In [ ]:
import numpy as np
import scipy.spatial as spatial
import pandas as pd
import pickle
import os
from smart_open import smart_open
import gensim

In [ ]:
%matplotlib inline

## To rebuild ground truth corpus from scratch (requires use of d2v scripts on repo)

In [ ]:
groundTruthFile='/Users/wrvhage/Research/EviDENce/GroundTruth/Manual_annotation_TOTAAL_door_Jeroen_en_Susan.xlsx'
groundTruthFileSheet='Resultaat Manuele Annotatie'
groundTruth = pd.read_excel(groundTruthFile, sheet_name=groundTruthFileSheet)


#groundTruth['Titel']


#type(groundTruth)

groundTruth.drop_duplicates(['Titel'],keep='first', inplace=True)
groundTruth.reset_index(drop=True,inplace=True)
#groundTruth['Titel'][121]


In [ ]:
stopwordsFile = '/Users/wrvhage/Research/EviDENce/ML/stopwords-nl-stopwords-iso_expanded_gensim_processed.txt'
corpus_dir = '/Users/wrvhage/Research/EviDENce/Data/NR-Teksts/EviDENce_NR_output_clean/TargetSize150'
corpus_token_file = 'GV_corpus_text_para_clean.txt'
corpus_text_file = 'GV_corpus_text_para_clean.txt'
corpus_filenames_file = 'GV_corpus_text_para_clean_filenames.txt'

full_ctokf = os.path.join(corpus_dir,corpus_token_file)
full_ctf = os.path.join(corpus_dir,corpus_text_file)
full_cff = os.path.join(corpus_dir,corpus_filenames_file) 

with open(stopwordsFile) as f:
    dutch_stopwords = f.readlines()
    

dutch_stopwords = [w.rstrip() for  w in dutch_stopwords]


def read_corpus(corpus_token_file,labeled=False):
    with smart_open(corpus_token_file,'r') as tf:
        for i,text_line in enumerate(tf):
            if labeled :
                #yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(text_line), [fn_line.rstrip()])
                yield gensim.models.doc2vec.TaggedDocument([token for token in gensim.utils.simple_preprocess(text_line,deacc=True) if token not in dutch_stopwords], [i])
            else:
                yield [token for token in gensim.utils.simple_preprocess(text_line,deacc=True) if token not in dutch_stopwords]

def read_corpus_lookup(corpus_filenames_file, corpus_token_file) :
    with smart_open(corpus_filenames_file, 'r') as fnf, smart_open(corpus_token_file,'r') as tf:
        i=0
        for (fn_line,tf_line) in zip(fnf,tf):
            yield ([i],[fn_line.rstrip()],[tf_line])
            i+=1


corp = list(read_corpus(full_ctokf,labeled=True))


corp_lookup = list(read_corpus_lookup(full_cff,full_ctokf))

reinf_corp_medvec= np.load('/Users/wrvhage/Research/EviDENce/ML/reinferred_corpus_terms_para_150_medvec_clean.npy')




In [ ]:
with open(full_cff,'r') as fnf:
    corpFileNames=fnf.readlines()

corpFileNames = [fn.rstrip().split('.')[0] for fn in corpFileNames ]
#corpFileNames = [fn.rstrip() for fn in corpFileNames ]


groundTruth_inCorpusIds =[]
for j in range(len(groundTruth)):
    #print(j, groundTruth['Titel'][j])
    gtTitel = groundTruth['Titel'][j]
    corpusIndex = [i for i,ent in enumerate(corpFileNames) if ent == gtTitel]
    groundTruth_inCorpusIds.append(corpusIndex[0])



#groundTruth_inCorpusIds




gt_corp_lookup = [corp_lookup[i] for i in groundTruth_inCorpusIds]



gt_corp=[corp[i] for i in groundTruth_inCorpusIds]


In [ ]:
gt_corp_reinf_medvec=reinf_corp_medvec[groundTruth_inCorpusIds]


### Append reinferred vectors to dataframe

series_reinfVec =pd.Series(np.array([]))
for i in range(len(gt_corp_reinf_medvec)):
    series_reinfVec[i] = gt_corp_reinf_medvec[i]
    


groundTruthEx = groundTruth



groundTruthEx.insert(10,'reinfVec',series_reinfVec)




## To import ground truth data set as pickle

In [ ]:
groundTruthEx = pd.read_pickle('/Users/wrvhage/Research/EviDENce/doc2vec/d2v_evaluation/groundTruthEx_pd.pkl')

In [ ]:
groundTruthEx


In [ ]:
def retrieval_pass(inVec, corpDF, nReturn):
    """calculate cosine distance of query vector inVec to all corpus vectors"""
    cosd_vec_corp = cosdistance_corp(inVec,corpDF['reinfVec'])
    
    """rank cosine distances in descending order"""
    ranked_retrieval = np.argsort(cosd_vec_corp)[::-1]
    
    "get cosine distances in ranked order"
    ret_cosd = cosd_vec_corp[ranked_retrieval]
    
    
    
    """ return top nReturn instances"""
    ranking = corpDF.iloc[ranked_retrieval,[1,9,10]].copy()
    ranking['cosine_distance']= ret_cosd
    

    
    selected = ranking.iloc[0:nReturn].copy()
    """ create pandas series of selected ranked cosine distances"""
    #ret_cosd = pd.Series(ranked_cosd[0:nReturn])
    
    """insert cosine distance into returned ranking"""
    
    
    return selected
    
    
    

In [ ]:
def cosdistance(v1,v2):
    d = np.array([1. - spatial.distance.cosine(v1,v2)])
    return d

In [ ]:
def cosdistance_corp(vec,corpus_vectors):
    cosd_vec_corp =[]
    for i in range (len(corpus_vectors)):
        cd = cosdistance(vec,corpus_vectors[i])
        cosd_vec_corp.append(cd)
    cosd_vec_corp = np.array(cosd_vec_corp).flatten()
    return cosd_vec_corp

In [ ]:
def calculate_PR_LR(retrievedDF):
    P_LR = len(np.array(np.where(retrievedDF['EINDOORDEEL_janee'] == 'ja')).flatten())/len(retrievedDF['EINDOORDEEL_janee'])
    R_LR = P_LR
    return P_LR, R_LR



In [ ]:
def run_evaluation(corpDF,lenRetrieve):

    precisionLR =[]
    recallLR =[]
    queries =[]
    
    loc_yes = np.array(np.where(corpDF['EINDOORDEEL_janee'] == 'ja')).flatten()
    
    for i in range(len(loc_yes)):
        queryVec = corpDF['reinfVec'][loc_yes[i]]
        queryTitle = corpDF['Titel'][loc_yes[i]]
        
        incorp = corpDF.copy()
        incorp.drop([loc_yes[i]],inplace=True)
        incorp = incorp.reset_index(drop=True)
        
        retrieved = retrieval_pass(queryVec,incorp,lenRetrieve)
        
        retrievedout = retrieved.drop(columns=['reinfVec'])#.to_csv(queryTitle+'_'+str(lenRetrieve)+'.csv',index=False)
        retrievedout.to_csv('/Users/wrvhage/Research/EviDENce/ML/evaluation_data/single_pass_retrieval/'+queryTitle+'_'+str(lenRetrieve)+'.csv',index=False)
        
        pLR, rLR = calculate_PR_LR(retrieved)
        
        precisionLR.append(pLR)
        recallLR.append(rLR)
        queries.append(queryTitle)
        
        
    return precisionLR, recallLR, queries

In [ ]:
sp_precisionAtR, sp_recallAtR, sp_queryTitle = run_evaluation(groundTruthEx,182)



In [ ]:
Data = {'Title': sp_queryTitle, 'PR_at_R': sp_precisionAtR }
results = pd.DataFrame(Data,columns=['Title','PR_at_R'])
results.sort_values(by='Title',inplace=True)



In [ ]:
results.to_csv('/Users/wrvhage/Research/EviDENce/ML/evaluation_data/single_pass_retrieval/evaluation_at_R.csv')



In [ ]:
sp_meanPaR = np.array(sp_precisionAtR).mean()
sp_stdPaR = np.array(sp_precisionAtR).std()
sp_maxPaR = np.array(sp_precisionAtR).max()
sp_minPaR = np.array(sp_precisionAtR).min()
sp_NAboveBaseline = len(np.array(np.where(np.array(sp_precisionAtR) > 0.1848)).flatten())
sp_FracAboveBaseline = sp_NAboveBaseline/183


In [ ]:
sp_meanPaR, sp_stdPaR, sp_maxPaR, sp_minPaR,sp_NAboveBaseline,sp_FracAboveBaseline

In [ ]:
182/985 #random sample based expectation

In [ ]:
def goldstandard_relevance_feedback(inVec, ranking, corpDF, k):
    # assume user has judged the top-k documents for relevance, neutrality and irrelevance
#    relevant_retrieved = np.array(np.where(ranking['EINDOORDEEL_janee'] == 'ja')).flatten()
#    irrelevant_retrieved = np.array(np.where(ranking['EINDOORDEEL_janee'] == 'nee')).flatten() # assume no neutral in manual annotations

#    print(type(relevant_retrieved))
#    print(relevant_retrieved)
    
#    relrank = ranking.loc[relevant_retrieved,['reinfVec']]
#    irrelrank = ranking.loc[irrelevant_retrieved,['reinfVec']]
    
    
    return relevance_feedback_pass(inVec, 
                                   ranking[ranking['EINDOORDEEL_janee'] == 'ja']['reinfVec'],
                                   ranking[ranking['EINDOORDEEL_janee'] == 'nee']['reinfVec'],
                                   corpDF,
                                   k)


In [ ]:
def relevance_feedback_pass(inVec, relevantVecs, irrelevantVecs, corpDF, k):
    # Rocchio relevance feedback
    relevantMeanVec = np.mean(relevantVecs, axis=0)
    irrelevantMeanVec = np.mean(irrelevantVecs, axis=0)

    a = 1
    b = 0.8
    c = 0.1
    
    updatedQueryVec = a * inVec + b * relevantMeanVec + c * irrelevantMeanVec
    
    return updatedQueryVec, retrieval_pass(updatedQueryVec, corpDF, 182)
    
    

In [ ]:
def run_feedback_evaluation(corpDF,lenRetrieve,lenSimUserFeedback):

    precisionLR =[]
    recallLR =[]
    queries =[]
    
    loc_yes = np.array(np.where(corpDF['EINDOORDEEL_janee'] == 'ja')).flatten()
    
    for i in range(len(loc_yes)):
        queryVec = corpDF['reinfVec'][loc_yes[i]]
        queryTitle = corpDF['Titel'][loc_yes[i]]
        
        incorp = corpDF.copy()
        incorp.drop([loc_yes[i]],inplace=True)
        incorp = incorp.reset_index(drop=True)
        
        ranking = retrieval_pass(queryVec,incorp,lenSimUserFeedback)
        updated_queryvec, retrieved = goldstandard_relevance_feedback(queryVec, ranking, incorp, lenSimUserFeedback)

        retrievedout = retrieved.drop(columns=['reinfVec'])#.to_csv(queryTitle+'_'+str(lenRetrieve)+'.csv',index=False)
        retrievedout.to_csv('/Users/wrvhage/Research/EviDENce/ML/evaluation_data/goldstandard_relevance_feedback_pass/'+queryTitle+'_'+str(lenRetrieve)+'.csv',index=False)
        
        pLR, rLR = calculate_PR_LR(retrieved)
        
        precisionLR.append(pLR)
        recallLR.append(rLR)
        queries.append(queryTitle)
        
        
    return precisionLR, recallLR, queries


In [ ]:
rf_precisionAtR, rf_recallAtR, rf_queryTitle = run_feedback_evaluation(groundTruthEx,182,20)


In [ ]:
Data = {'Title': rf_queryTitle, 'PR_at_R': rf_precisionAtR }
results = pd.DataFrame(Data,columns=['Title','PR_at_R'])
results.sort_values(by='Title',inplace=True)



In [ ]:
results.to_csv('/Users/wrvhage/Research/EviDENce/ML/evaluation_data/goldstandard_relevance_feedback_pass/evaluation_at_R.csv')



In [ ]:
rf_meanPaR = np.array(rf_precisionAtR).mean()
rf_stdPaR = np.array(rf_precisionAtR).std()
rf_maxPaR = np.array(rf_precisionAtR).max()
rf_minPaR = np.array(rf_precisionAtR).min()
rf_NAboveBaseline = len(np.array(np.where(np.array(rf_precisionAtR) > 0.1848)).flatten())
rf_FracAboveBaseline = rf_NAboveBaseline/183


In [ ]:
rf_meanPaR, rf_stdPaR, rf_maxPaR, rf_minPaR, rf_NAboveBaseline, rf_FracAboveBaseline

In [ ]:
182/985 # random sample expectation

In [ ]:
((np.array(rf_precisionAtR) - np.array(sp_precisionAtR)) > 0).sum()
